In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func

spark = SparkSession.builder.appName("Reading from sockets").master("local[*]").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 8)

spark

In [2]:
# Read stream
df_raw = spark.readStream.format("socket").option("host", "localhost").option("port", "9999").load()

In [3]:
# Split line into array
df_words = df_raw.withColumn("words", func.split("value", " "))

In [4]:
# Explode array into new row and delete other columns
df_word = df_words.withColumn("word", func.explode(df_words.words)).drop("value", "words")

In [5]:
# Agg data and count words
df_words_count = df_word.groupBy("word").agg(func.count("word").alias("count"))

In [ ]:
df_words_count.writeStream.format("console").outputMode("complete").start().awaitTermination()